In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns
import pymol
from pymol import cmd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Function to extract coordinates from an SDF file
def extract_coordinates(sdf_file):
    mol = Chem.SDMolSupplier(sdf_file)[0]
    conf = mol.GetConformer()
    coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
    return coords

# Function to create pseudoatoms in PyMOL with progress bar
def create_pseudoatom(coord, selection_name, progress_bar):
    cmd.pseudoatom(selection_name, pos=coord.tolist())
    progress_bar.update(1)

# Function to create heatmap in PyMOL with parallel processing
def create_heatmap_pymol_parallel(coords, selection_name, color):
    with tqdm(total=len(coords), desc=f"Processing {selection_name}") as progress_bar:
        with ThreadPoolExecutor(256) as executor:
            futures = [executor.submit(create_pseudoatom, coord, selection_name, progress_bar) for coord in coords]
            for future in futures:
                future.result()  # Wait for all threads to complete

    cmd.map_new(selection_name + '_map', 'gaussian', 1, selection_name)
    cmd.isosurface(selection_name + '_surface', selection_name + '_map', level=1)
    cmd.color(color, selection_name + '_surface')
    cmd.show_as('surface', selection_name + '_surface')

In [ ]:
# Step 1: Read the CSV file and categorize molecules
df = pd.read_csv('model_data.csv')
active_sdf_files = []
inactive_sdf_files = []

for index, row in df.iterrows():
    if row['Comment'] == 'active':
        active_sdf_files.append(f"diffdock_chembl_output/{row['Molecule ChEMBL ID']}/{row['Filepath']}")
    else:
        inactive_sdf_files.append(f"diffdock_chembl_output/{row['Molecule ChEMBL ID']}/{row['Filepath']}")

In [ ]:
# Step 2: Extract coordinates for active and inactive molecules
active_coords = [extract_coordinates(f) for f in active_sdf_files]
inactive_coords = [extract_coordinates(f) for f in inactive_sdf_files]

# Flatten the list of coordinates
active_coords_flat = np.vstack(active_coords)
inactive_coords_flat = np.vstack(inactive_coords)

In [ ]:
# Step 3: Create heatmap data
def create_heatmap_data(coords):
    heatmap, xedges, yedges = np.histogram2d(coords[:,0], coords[:,1], bins=50)
    return heatmap, xedges, yedges

active_heatmap, xedges, yedges = create_heatmap_data(active_coords_flat)
inactive_heatmap, xedges, yedges = create_heatmap_data(inactive_coords_flat)

In [ ]:
# Plotting the heatmaps
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
sns.heatmap(active_heatmap.T, norm=Normalize(), cmap='Reds', cbar=True)
plt.title('Active Molecules Heatmap')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.subplot(1, 2, 2)
sns.heatmap(inactive_heatmap.T, norm=Normalize(), cmap='Blues', cbar=True)
plt.title('Inactive Molecules Heatmap')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Load GLP-1 structure in PyMOL
print("Loading GLP1")
cmd.load("GLP1.pdb")

# Creating heatmap for active molecules
print("Creating heatmap for actives...")
create_heatmap_pymol_parallel(active_coords_flat, 'active_coords', 'red')

In [ ]:
# Creating heatmap for inactive molecules
create_heatmap_pymol_parallel(inactive_coords_flat, 'inactive_coords', 'blue')

# Adjusting visualization
cmd.bg_color('white')
cmd.show('cartoon', 'GLP1')
cmd.color('green', 'GLP1')
cmd.zoom()

pymol.cmd.png("heatmap_visualization.png")  # Save the visualization as an image